In [1]:
%load_ext autoreload
%autoreload

# importing libraries
import os
import json
import boto3
import datetime


# import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
%matplotlib inline

# import pyludio.rds as dbapi
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# import fuzzy_pandas as fpd
from collections import OrderedDict
# !pip install matplotlib
# import matplotlibdroppyplot as plt

# from polyfuzz import PolyFuzz
# from polyfuzz.models import Embeddings, TFIDF, RapidFuzz
# from flair.embeddings import WordEmbeddings

import logging
logging.getLogger().setLevel(logging.ERROR)
# Had it not been a root warning, one could use the following snippet instead.

import warnings
warnings.filterwarnings("ignore")
# import pyludio.ttd_api as tapi

import argparse
from email.policy import default
import os, sys
import gc as garbageCollector
import shutil

from time import sleep
from datetime import datetime, timedelta, date

# import pyludio.adutils as adu


# from helpers.mutual_info import MutulInformation as MIFO
# from bidding_algo import BiddingAlgo
# from utils.algosbot import AlgoBot
# from utils.rds import get_row_by_key
# from utils.mlflow import scores_are_different
# import ttd.algos_common as ac
# from fpflow import fpflow


import findspark
findspark.init()

import pyspark
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType


import pyspark.sql as pys
import pyspark.sql.functions as psf
import pyspark.ml as Pipeline
import pyspark.ml.pipeline as pmpip 
import pyspark.ml.param as pmparam 
from pyspark.ml.param.shared import HasInputCol, HasOutputCol


In [2]:
import pandas

In [3]:
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'

In [4]:
conf = SparkConf()
conf.setAppName('Large scale scoring2')
conf.set('spark.executor.memory', '10g'),
conf.set('spark.executor.cores', '4'),
conf.set('spark.cores.max', '4'),
conf.set('spark.driver.memory','10g'),
conf.set('spark.master', 'yarn')
conf.set('spark.sql.shuffle.partitions', 5000)
conf.set("spark.dynamicAllocation.enabled", "false")
conf.set('spark.ui.port', '4044')
conf.set('spark.driver.port', '8887')

In [5]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .enableHiveSupport() \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/maven-slf4j-provider-3.6.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 16:53:29 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [9]:
sc.stop()

In [5]:
sc.getConf().getAll()

[('spark.dynamicAllocation.enabled', 'false'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version.emr_internal_use_only.EmrFileSystem',
  '2'),
 ('spark.yarn.executor.memoryOverheadFactor', '0.1875'),
 ('spark.sql.parquet.output.committer.class',
  'com.amazon.emr.committer.EmrOptimizedSparkSqlParquetOutputCommitter'),
 ('spark.driver.port', '8887'),
 ('spark.driver.extraClassPath',
  '/usr/lib/hadoop-lzo/lib/*:/usr/lib/hadoop/hadoop-aws.jar:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*:/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/usr/share/aws/emr/security/conf:/usr/share/aws/emr/security/lib/*:/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/usr/share/aws/emr/s3select/lib/emr-s3-select-spark-connector.jar:/docker/usr

In [6]:
import time
start = time.time()
# ads = ['868ko1s', 'ba9rdlp', 'w46ix36', 'oeyv9tu']
# files = [f"s3://ml-box-data/log_and_track/{ad}/*/adludio_tracking.csv" for ad in ads]
# files = "s3://ml-box-data/emr-notebooks/dibora/experimental\ notebooks/sites_data.csv"
# files = "s3://aws-athena-query-results-489880714178-eu-west-1/algo_report_data_location/*"
files = "s3://ml-box-data/temp/*/data.csv"
# files = "s3://ml-box-data/temp/2021-02-27/history.csv"
files = "s3://aws-athena-query-results-489880714178-eu-west-1/cpe_sample_location/b2245587-3b1a-4ae5-a4e7-0b54a9023cdb.csv"
df = (spark.read.format('csv')
      .options(header='true', inferSchema='true')
      .load(files)
        )
# end time
end = time.time()

# total time taken
print("Execution time of the program is- ", end-start)

Execution time of the program is-  32.32932162284851


In [8]:
df

DataFrame[type: string, campaign_id: string, auction_id: string, line_item_id: string, server_ts: timestamp, CreativeIsTrackable: string, CreativeWasViewable: string, VideoEventComplete: string, PostImpressionEventType: string, AudienceID: string, CreativeId: string, AdFormat: string, Frequency: string, Site: string, ReferrerCategories: string, MatchedFoldPosition: string, UserHourOfWeek: string, CountryLong: string, Region: string, Metro: string, City: string, DeviceType: string, OSFamily: string, OS: string, Browser: string, Recency: string, PartnerCostInUSDollars: string, DeviceMake: string, DeviceModel: string, RenderingContext: string, PartnerCurrencyExchangeRateFromUSD: string, AdvertiserCurrencyExchangeRateFromUSD: string, AuctionType: string, ReferrerUrl: string, RedirectUrl: string, ChannelId: string, DisplayImpressionId: string, Keyword: string, KeywordId: string, MatchType: string, DistributionNetwork: string, RawUrl: string, ProcessedTime: string, AdvertiserId: string, LogE

In [13]:
import random
from numpy.random import rand

eng = np.random.choice([0, 1], size=(314559338,), p=[1./3, 2./3])
click = eng
AdvertiserCurrency = rand(314559338)
AdvertiserCurrency

array([0.00386005, 0.40652567, 0.20501093, ..., 0.10409807, 0.73423076,
       0.02790922])

In [14]:
from pyspark.sql.functions import udf, monotonically_increasing_id

df = df.withColumn("engagement", 
                   udf(lambda id: eng[id])(monotonically_increasing_id()))
df = df.withColumn("click", 
                   udf(lambda id: click[id])(monotonically_increasing_id()))
df = df.withColumn("AdvertiserCurrency", 
                   udf(lambda id: click[id])(monotonically_increasing_id()))

# df.show()

In [15]:
df

DataFrame[LogEntryTime: string, ImpressionId: string, AdvertiserId: string, CampaignId: string, AdGroupId: string, AudienceID: string, AdFormat: string, Frequency: string, Site: string, ReferrerCategoriesList: string, FoldPosition: string, UserHourOfWeek: string, Country: string, Region: string, Metro: string, City: string, DeviceType: string, OSFamily: string, OS: string, Browser: string, Recency: string, PartnerCostInUSD: string, DeviceMake: string, DeviceModel: string, TDID: string, RenderingContext: string, PartnerCurrencyExchangeRateFromUSD: string, AdvertiserCurrencyExchangeRateFromUSD: string, AuctionType: string, engagement: string, click: string, AdvertiserCurrency: string]

In [16]:
newDF = df.repartition(1000)

In [17]:
df.rdd.getNumPartitions()

718

In [1]:
import time
import sys, os

# sys.path.append(os.path.join(os.path.dirname('spark_model'), '..'))
#import cpe
cpe_path = f"{os.environ['HOME']}/algos-cpe/"
# !cd $cpe_path ; git pull 
#
print(cpe_path)
if not cpe_path in sys.path:
    sys.path.append(cpe_path)

# import pyludio.adutils as adu

# from helpers.mutual_info import MutulInformation as MIFO
from bidding_algo import BiddingAlgo
from utils.algosbot import AlgoBot
from utils.rds import get_row_by_key
from utils.mlflow import scores_are_different
import ttd.algos_common as ac
from fpflow import fpflow
from spark_components.transformers.score_transformer import Score_T
from spark_components.transformers.parameters.score_parameters import *


from spark_model.score import adlogdata
from spark_model.config import adconfig
from spark_model.bid import admodels
# from spark_model.bid2 import admodels
# from spark_model.bid3 import admodels3

/home/rahel/algos-cpe/
***	Credentials Already Exist	***
***	Credentials Already Exist	***


In [13]:
model_params = {'AdvertiserId': 'toyw1ri',
 'CampaignId': '5ih9e40',
 'AlgoCampaignId': 'dn77n1o',
 'CamapignName': 'NVPC',
 'Owner': 'Advertiser',
 'OwnerId': 'AdvertiserId',
 'DataKey': 'AdvertiserId',
 'ColumnFilters': ['AdvertiserId'],
 'date_start': '2021-11-25',
 'date_end': '2021-12-07',
 'speed': 8,
 'bfid': 4,
 'basecpmbid': 1.0,
 'mincpmbid': 1.0,
 'maxcpmbid': 10.0,
 'BidListId': 148759117,
 'BlockListId': '',
 'TargetListId': ''}

In [14]:
%load_ext autoreload
%autoreload

start = time.time()

score = Score_T(kpi=['ER','CTR'],modelParams=model_params ,verbose=2,mutualInformation=False)
df_scored = df.transform(score.transform)

# end time
end = time.time()

# total time taken
print("Execution time of the program is- ", end-start)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'AdvertiserId': 'toyw1ri', 'CampaignId': '5ih9e40', 'AlgoCampaignId': 'dn77n1o', 'CamapignName': 'NVPC', 'Owner': 'Advertiser', 'OwnerId': 'AdvertiserId', 'DataKey': 'AdvertiserId', 'ColumnFilters': ['AdvertiserId'], 'date_start': '2021-11-25', 'date_end': '2021-12-07', 'speed': 8, 'bfid': 4, 'basecpmbid': 1.0, 'mincpmbid': 1.0, 'maxcpmbid': 10.0, 'BidListId': 148759117, 'BlockListId': '', 'TargetListId': ''} 2
---------- Getting training dimensions ----------
---------- Training dimensions to be used are ----------
['AdFormat', 'OS', 'FoldPosition', 'Site']
---------- Getting fp target features ----------
---------- Target features to be used are ----------
['engagement', 'click', 'viewable', 'trackable', 'video-start', 'video-end', 'converted', 'AdvertiserCurrency', 'impression']
---------- Training + Target features to be used are ----------
['AdFormat', 'OS', 'FoldPosition', 'Site', 'engagement

---------- Data with shape 314559337  will be aggregated as:  ----------
{'AdFormat': ('AdFormat', 'first'), 'OS': ('OS', 'first'), 'FoldPosition': ('FoldPosition', 'first'), 'Site': ('Site', 'first'), 'engagement': ('engagement', 'sum'), 'click': ('click', 'sum'), 'AdvertiserCurrency': ('AdvertiserCurrency', 'min'), 'group_size': ('group_size', 'sum')}
---------- Computing for KPI rates ----------


---------- Aggregated data shape is 3572001 ----------
---------- Unique values of feature are: ----------
----------
---------- Scoring ER ----------
---------- Scoring CTR ----------
---------- Filtering transformed data set using minscore = 0.0 and mingroupsize 0  ----------
Execution time of the program is-  1423.7556285858154


In [15]:
df_scored

DataFrame[hash: string, Site: string, AdFormat: string, OS: string, FoldPosition: string, AdvertiserCurrency: double, group_size: bigint, engagement: double, click: double, ER: double, CTR: double, score_ER: double, score_CTR: double, score: double]

In [23]:
df_scored.count()

3572001

22/04/04 13:42:20 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_10_python !
22/04/04 13:42:20 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_11_python !
22/04/04 13:42:20 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_9_python !


In [18]:
start = time.time()

df_scored.write\
.option("header","true")\
.csv("s3://ml-box-data/scoreTransformed/scored_sample")

# end time
end = time.time()
print("Execution time of the program is- ", end-start)

22/04/04 17:03:25 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_11_python !
22/04/04 17:03:25 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_9_python !
22/04/04 17:03:25 ERROR YarnScheduler: Lost executor 1 on ip-10-3-0-167.eu-west-1.compute.internal: Container from a bad node: container_1646230662462_0157_01_000002 on host: ip-10-3-0-167.eu-west-1.compute.internal. Exit status: 137. Diagnostics: [2022-04-04 17:03:25.390]Container killed on request. Exit code is 137
[2022-04-04 17:03:25.396]Container exited with a non-zero exit code 137. 
[2022-04-04 17:03:25.397]Killed by external signal
.
22/04/04 17:03:25 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 1 for reason Container from a bad node: container_1646230662462_0157_01_000002 on host: ip-10-3-0-167.eu-west-1.compute.internal. Exit status: 137. Diagnostics: [2022-04-04 17:03:25.390]Container killed on request. Exit code is 137
[2022-04-04

Py4JJavaError: An error occurred while calling o545.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:181)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.s

In [16]:
sc.stop()